In [21]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [22]:
load_dotenv()
mapbox_token = os.getenv("mapbox")

In [23]:
file_path = Path("./Desktop/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("./Desktop/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

In [24]:
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_mean_per_year = sfo_data.groupby("year").mean()
    sfo_data_mean_per_year.reset_index(inplace=True)

    std=sfo_data_mean_per_year["housing_units"].std()
    mins=sfo_data_mean_per_year["housing_units"].min()-std
    maxes=sfo_data_mean_per_year["housing_units"].max()+std

    housing_units_per_year_fig = px.bar(
    sfo_data_mean_per_year,
    x="year",
    y="housing_units",
    range_y=[mins,maxes],
    labels={"year":"Year","housing_units":"Housing Units"},
    title="Housing Units in San Francisco from 2010 to 2016",
    width=700, 
    height=600
    )
    return housing_units_per_year_fig

In [25]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_mean_per_year = sfo_data.groupby("year").mean()
    sfo_data_mean_per_year.reset_index(inplace=True)

    average_gross_rent_per_year_fig = px.line(
    sfo_data_mean_per_year,
    x="year",
    y="gross_rent",
    labels={"year":"Year","gross_rent":"Gross Rent"},
    title="Average Gross Rent in San Francisco",
    width=700, 
    height=600
    )
    return average_gross_rent_per_year_fig

In [26]:
def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    sfo_data_mean_per_year = sfo_data.groupby("year").mean()
    sfo_data_mean_per_year.reset_index(inplace=True)

    average_sale_price_per_squre_foot_fig = px.line(
    sfo_data_mean_per_year,
    x="year",
    y="gross_rent",
    labels={"year":"Year","gross_rent":"Gross Rent"},
    title="Average Gross Rent in San Francisco",
    width=700, 
    height=600
    )
    return average_sale_price_per_squre_foot_fig

In [27]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    average_prices_by_neighborhood = sfo_data.groupby(["year","neighborhood"]).mean()
    average_prices_by_neighborhood.reset_index(inplace=True)
    average_prices_by_neighborhood_plot = average_prices_by_neighborhood.hvplot.line(x='year',y="sale_price_sqr_foot",ylabel='Avg. Sale Price per Square Foot',groupby='neighborhood')
    return average_prices_by_neighborhood_plot


In [28]:
def mean_neighborhood_top_10():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot = mean_neighborhood_top_10.hvplot.bar(x='neighborhood',y="sale_price_sqr_foot",ylabel='Avg. Sale Price per Square Foot',rot=90,width=800,height=460)
    return mean_neighborhood_top_10_plot

In [29]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot_parra_coor = px.parallel_coordinates(mean_neighborhood_top_10, color='sale_price_sqr_foot')
    return mean_neighborhood_top_10_plot_parra_coor

In [30]:
def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    mean_neighborhood = sfo_data.groupby("neighborhood").mean()
    mean_neighborhood = mean_neighborhood.sort_values(by="sale_price_sqr_foot", ascending=False)
    mean_neighborhood_top_10 = mean_neighborhood.head(10)
    mean_neighborhood_top_10.reset_index(inplace=True)
    mean_neighborhood_top_10_plot_parra_cat = px.parallel_coordinates(mean_neighborhood_top_10, color='sale_price_sqr_foot')
    return mean_neighborhood_top_10_plot_parra_cat

In [31]:
def neighborhood_map():
    """Neighborhood Map"""
    df_neighborhood_locations.rename(columns={"Neighborhood": "neighborhood"}, inplace=True)
    df_neighborhood_locations.head()
    neighborhood = sfo_data.groupby("neighborhood").mean()
    neighborhood.reset_index(inplace=True)
    mean_neighborhood_location = pd.merge(df_neighborhood_locations, neighborhood, on='neighborhood')
    mean_neighborhood_location_map = px.scatter_mapbox(mean_neighborhood_location, title="Averange Sale Price Per Square Foot and Gross Rent in San Francisco",lat="Lat", lon="Lon", size='sale_price_sqr_foot',color="gross_rent",zoom=10,hover_name='neighborhood')
    return mean_neighborhood_location_map

In [54]:
Intro = pn.pane.Markdown("San Francisco  property data from 2010 to 2016")

tab_1_columns = pn.Column(
    (" San Francisco Housing Rental Analysis - 2010 to 2016"),
    (" Housing Units Per Year"),
    housing_units_per_year,
    (" Average Gross Rent Per Year"),
    average_gross_rent,
    (" Average Sales Price Per Year"),
    average_sales_price
)

tab_2_columns = pn.Column(
    (" San Francisco Neighborhood Analysis - 2010 to 2016"),
    (" Average Prices by Neighborhood"),
    average_price_by_neighborhood,
    (" Top 10 Most Expensive Neighborhoods"),
    mean_neighborhood_top_10
)

tab_3_columns = pn.Column(
    (" Parallel Coordinates and Parallel Categories Analysis"),
    (" Parallel Coordinates Plot"),
    parallel_coordinates,
    (" Parallel Categories Plot"),
    parallel_categories
)




# Create dashboard Tabs
panel = pn.Tabs(
    ("Intro", Intro),
    ("Yearly Market Analysis", tab_1_columns),
    ("Neighborhood Analysis", tab_2_columns),
    ("Parallel Plots Analysis",tab_3_columns),
)

In [55]:
panel.servable()

Tabs
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07749')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07757')
        [5] Markdown(str)
        [6] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07765')
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive07776')
                    [0] HoloViews(DynamicMap, name='interactive07776')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive07816')
    [3] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07906')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive07914')